### Import Necessary Library

In [1]:
import pinecone
import numpy as np
# from tqdm.autonotebook import tqdm
from transformers import AutoTokenizer, AutoModel
from haystack.document_stores import PineconeDocumentStore

c:\Users\uqba2\miniconda3\envs\ml\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


* Define the name of your index

In [2]:
index_name = 'semantic-search-ai'
document_store = pinecone.init(
    api_key="a88f41cf-3264-48d8-8a04-953f06214d55",
    environment = "us-central1-gcp"
)

* Create an index

In [3]:
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=len(embeds[0]))
# connect to index
index = pinecone.Index(index_name)

###### `vblagoje/wikipedia_snippets_streamed` is a public dataset available on the Hugging Face Datasets platform. It contains a large collection of Wikipedia snippets, which are short summaries of Wikipedia articles, extracted from a dump of the English Wikipedia. The dataset includes over 5 million snippets, covering a wide range of topics and domains.

###### You can access the dataset on the Hugging Face Datasets website at [https://huggingface.co/datasets/vblagoje/wikipedia_snippets_streamed](https://huggingface.co/datasets/vblagoje/wikipedia_snippets_streamed). The website provides detailed information about the dataset, including the format, size, and license.

###### To use the dataset in Python, you can install the `datasets` library from Hugging Face and load the dataset using its name:

* Loading Dataset

In [4]:
from datasets import load_dataset

In [5]:
data = load_dataset('vblagoje/wikipedia_snippets_streamed',
                    split='train',
                    streaming=True)
data

###### The dataset is streamed, which means that the examples are not loaded into memory all at once. Instead, they are provided one by one as you iterate over the dataset. This makes it memory-efficient and suitable for large-scale natural language processing tasks.

In [21]:
filtered_dataset = data.filter(lambda d:d['section_title'].startswith('History'))

In [18]:
from haystack.nodes import EmbeddingRetriever
from haystack.nodes import DensePassageRetriever

In [13]:
retriever = DensePassageRetriever(document_store=document_store,
                                 query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                 passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                 use_gpu=False)

c:\Users\uqba2\miniconda3\envs\ml\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [19]:
ret = EmbeddingRetriever(
    document_store= document_store,
    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
    model_format='sentence_transformers',
)

In [14]:
from haystack import Document
from tqdm.auto import tqdm

In [17]:
# total_doc_count = 50000
# batch_size = 256

# counter = 0
# docs=[]
# for d in tqdm(df, total=total_doc_count):
#     doc= Document(
#         content=d['passage_text'],
#         meta={
#             'article_title':d['article_title'],
#             'section_title':d['section_title'],
#         }
#     )
#     docs.append(doc)
#     counter += 1
#     if counter % batch_size==0:
#         embeds = retriever.embed_documents(docs)
#         for i , doc in enumerate(docs):
#             doc.embedding = embeds[i]
#         document_store.write_documents(docs)
#         docs.clear()
#     if counter == total_doc_count:
#         break    

  0%|          | 0/50000 [00:00<?, ?it/s]

Create embeddings:   0%|          | 0/256 [00:00<?, ? Docs/s]

AttributeError: 'NoneType' object has no attribute 'write_documents'

In [22]:
# Create haystack document objects and embeddings
docs = []
embeddings = []
for d in tqdm(filtered_dataset):
    doc = Document(
        content=d['passage_text'],
        meta={
            'article_title': d['article_title'],
            'section_title': d['section_title'],
        }
    )
    docs.append(doc)
    embeddings.append(retriever.embed_passages([d['passage_text']])[0])

# Add documents and embeddings to the document store
document_store.write_documents(docs)
document_store.update_embeddings(embeddings, ids=list(range(len(docs))))

0it [00:00, ?it/s]

AttributeError: 'DensePassageRetriever' object has no attribute 'embed_passages'

In [ ]:
# total_doc_count = 50000
# batch_size = 256

# docs = []
# for i, d in enumerate(tqdm(df, total=total_doc_count)):
#     doc = Document(
#         content=d['passage_text'],
#         meta={
#             'article_title': d['article_title'],
#             'section_title': d['section_title'],
#         }
#     )
#     docs.append(doc)
#     if (i + 1) % batch_size == 0 or (i + 1) == total_doc_count:
#         embeds = retriever.embed_documents(docs)
#         for j, doc in enumerate(docs):
#             doc.embedding = embeds[j]
#         document_store.write_documents(docs)
#         docs.clear()


In [ ]:
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

: 

In [ ]:
search_pipeline = DocumentSearchPipeline(retriever)
result = search_pipeline.run(
    query = "When was the first electric power system built?",
    params={'retriever':{'top_k':2}}
)
print_documents(result)

: 

* Initializing the generator

In [ ]:
from haystack.nodes import Seq2SeqGenerator

: 

In [ ]:
generator = Seq2SeqGenerator(model_name_or_path='vblagoje/bart_lfqa')

: 

* Initializing a Generative QA Pipeline

In [ ]:
from haystack.pipelines import GenerativeQAPipeline
pipe = GenerativeQAPipeline(generator, ret)

: 

* Asking Questions

In [ ]:
result = pipe.run(
    query="what was the war of currents?",
    params={
        'Retriever':{'top_k':3},
        'Generator':{'top_k':1},
    }
)
result

: 

In [ ]:
result = pipe.run(
    query="what was the war of currents?",
    params={
        'Retriever':{'top_k':3},
        'Generator':{'top_k':1},
    }
)
print_documents(result, details='minimum')

: 

In [ ]:
result = pipe.run(
    query="what was the war of currents?",
    params={
        'Ret':{'top_k':10},
        'generator':{'top_k':1},
    }
)
print_documents(result, details='minimum')

: 

In [ ]:
result = pipe.run(
    query="What is the capital of France?",
    params={
        'Ret':{'top_k':3},
        'generator':{'top_k':1},
    }
)
print_documents(result, details='minimum')

: 

: 

: 

: 

In [ ]:
# Define a function to generate embeddings for a list of documents
def generate_embeddings(documents):
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
    model = AutoModel.from_pretrained("sentence-transformers/LaBSE")

    embeddings = []
    for doc in documents:
        inputs = tokenizer(doc, padding=True, truncation=True, max_length=512, return_tensors="pt")
        outputs = model(**inputs)
        embedding = np.array(outputs[1].detach().numpy()[0])
        embeddings.append(embedding)

    return embeddings

: 

In [ ]:
# Define a function to generate an embedding for a user query
def generate_query_embedding(query):
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
    model = AutoModel.from_pretrained("sentence-transformers/LaBSE")

    inputs = tokenizer(query, padding=True, truncation=True, max_length=512, return_tensors="pt")
    outputs = model(**inputs)
    embedding = np.array(outputs[1].detach().numpy()[0])

    return embedding

: 

In [ ]:
# Define a function to search for similar documents
def search_similar_documents(query_embedding, top_k=10):
    result = index.query(queries=[query_embedding], top_k=top_k)
    return result.ids[0], result.scores[0]

: 

In [ ]:
# Generate embeddings for your documents
documents = data['text']
document_embeddings = generate_embeddings(documents)

: 

In [ ]:
# Upload embeddings to the index
index.upsert(ids=data['id'], embeddings=document_embeddings)

: 

In [ ]:
# Example usage
query = "search query"
query_embedding = generate_query_embedding(query)
similar_docs_ids, similar_docs_scores = search_similar_documents(query_embedding)
print(similar_docs_ids)
print(similar_docs_scores)

: 